In [ ]:
from gradio_client import Client
from pathlib import Path
import json
import pandas as pd

In [14]:
path_data = Path("../data")

In [ ]:
client = Client("https://felixz-open-llm-leaderboard.hf.space/")
json_data = client.predict("","", api_name='/predict')

with open(json_data, 'r') as file:
    file_data = file.read()
    data = json.loads(file_data)
    df = pd.DataFrame(data['data'], columns=data['headers'])
    df.drop(columns=['Model'], inplace=True)
    df.to_json(path_data / 'HF-open-llm-leaderboard_20231114.json', orient='records', indent=4)

In [64]:
client = Client("https://upstage-open-ko-llm-leaderboard.hf.space/")
json_data = client.predict("","", api_name='/predict')

with open(json_data, 'r') as file:
    file_data = file.read()
    data = json.loads(file_data)
    df = pd.DataFrame(data['data'], columns=data['headers'])
    df.drop(columns=['Model'], inplace=True)
    df.to_json(path_data / 'HF-open-ko-llm-leaderboard_20231114.json', orient='records', indent=4)

Loaded as API: https://upstage-open-ko-llm-leaderboard.hf.space/ ✔


ValueError: Cannot find a function with `api_name`: /predict.

In [63]:
df = pd.read_csv('b.csv')
df.to_json(path_data / 'SuperCLUE-safety-leaderboard-指令攻击安全-20231108.json', orient='records', indent=4)